## Import necessary libraries

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Black-Scholes Model
<p>
    <font size=4>
        The Black-Scholes Model is used for pricing the Vanilla European Options, its formula is as follow:
<p/>
<p style='text-align: center;'>
    <font size=4>
        $C=S_0N(d_1)-Ke^{-rT}N(d_2)$  <br>
        $P=Ke^{-rT}N(-d_2)-S_0N(-d_1)$  <br>
        where $d_1=\frac{ln\frac{S_0}{K}+(r+\frac{\sigma^2}{2})T}{\sigma\sqrt T}$ <br>
        $d_2=d_1-\sigma\sqrt T$ <br>
        $S_0$: spot price <br>
        $K$: strike price <br>
        $r$: risk-free rate <br>
        $q$: dividend yield <br>
        $\sigma$: annual volatility <br>
        $T$: year-to-maturity <br>
        $N(.)$: cdf of a standard normal
<p/>
<p>
    <font size=4>
        Note the Black-Scholes Formula is only applicable to the European Options. And it gives the most accurate value since it is an analytic formula.
<p/>

In [5]:
def BS_Euro_Pricer(S, K, r, q, T, sig, isCall=True):
    """
    This function finds the Black Scholes price of an option
    input:
    S: Underlying asset spot price
    K: Strike price
    r: risk-free rate (if r = 4%, input 0.04)
    q: continuous dividend yield (if q = 4%, input 0.04) 
    T: Year to maturity
    sig: standard deviation of stock return (if sig = 20%, input 0.2)
    isCall: True if call option, vice versa
    """
    d1 = (np.log(S/K) + (r - q + (sig**2) / 2) * T) / (sig * np.sqrt(T))
    d2 = d1 - sig * np.sqrt(T)
    if isCall:
        return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

In [3]:
BS_Euro_Pricer(100, 100, .05, 0, 1, .25)

12.335998930368717

## Binomial Tree
<p>
    <font size=4>
        The Binomial Tree is another model for derivative pricing. It can be used to price both European and American Options, its logic is as follow:
<p/>
<p style='text-align: center;'>
    <font size=4>
        Denote $i=1,2,...,n-1$ such that $t_i=\frac{T}{n}i$ where $n$ is the number of steps of the tree and $T$ is the Year-to-maturity<br>
        Denote $dt=\frac{T}{n}$, for each $t_i$, we have $S_{t_{i+1},u}=S_{t_i}u$, and $S_{t_{i+1},d}=S_{t_i}d$  <br>
        where $u=e^{\sigma dt}$, $d=\frac{1}{u}$ <br>
        $S_{t_0}$: spot price <br>
        $\sigma$: annual volatility <br><br>
        Since it is a recombinant tree, if we have $n$ steps, there will be $n+1$ terminal prices. <br>
        Denote $k=1,2,...,n+1$ such that $S_{k,n+1}$ is the terminal price in the {k}th case <br>
        Take European Call as example, we have $C_{k,n+1}=(S_{k,n+1}-K)^+$ where $K$ is the strike price <br>
        Take 1 step backwards, denote $j=1,2,...,n$ such that $C_{j,n}$ is the Call price in the {j}th case at time $T-\frac{1}{n}$ <br>
        We have $C_{j,n}=e^{-rdt}(p_uC_{j,n+1}+p_dC_{j+1,n+1})$ <br>
        where $p_u=\frac{e^{(r-q)dt}-d}{u-d}$ and $p_d=1-p_u$<br>      
        $r$: risk-free rate <br>
        $q$: the dividend yield <br>
        Keep computing the Call price from backwards, we can eventually get $C_0$ <br> <br>
        For American Call, denote $c_{j,n}$ is the American Call price in the {j}th case at time $T-\frac{1}{n}$<br>
        Before we move 1 step backwards, we need to compute the value of early exercise $V$ and take $c_{j,n}=max(V,C_{j,n})$ <br>
        Then, the remaining steps are same as calculating $C_0$
<p/>
<p>
    <font size=4>
        Note the Binomial Tree is applicable to the both European and American Options because it is backward looking
<p/>

In [6]:
def Binomial_Pricer(S, K, r, q, T, sig, n, isEuropean=True, isCall=True):
    """
    This function finds the Binomial price of an option
    input:
    S: Underlying asset spot price
    K: Strike price
    r: risk-free rate (if r = 4%, input 0.04)
    q: continuous dividend yield (if q = 4%, input 0.04) 
    T: Year to maturity
    sig: standard deviation of stock return (if sig = 20%, input 0.2)
    n: steps of the tree
    isEuropean: True if European option, vice versa
    isCall: True if call option, vice versa
    """
    dt = T / n # Delta t
    u = np.exp(sig * np.sqrt(dt)) 
    d = 1 / u
    P_u = (np.exp((r - q) * dt) - d) / (u - d)
    P_d = 1 - P_u
    S_T = S * (u ** np.arange(n, -1, -1)) * (d ** np.arange(0, n+1)) # Terminal values of stock price
    
    # Calculate call/ put payoff
    if isCall:
        C = np.maximum(S_T - K, np.zeros(n+1))
    else:
        C = np.maximum(K - S_T, np.zeros(n+1))
    if isEuropean:
        for i in range(n, 0, -1): # Calculate value of option from backwards, last value is 1, which is value at time 0
            C_u = C[0:i] # Top i value
            C_d = C[1:i+1] # Bottom i value
            C = (P_u * C_u + P_d * C_d) / np.exp(r * dt) # nrow of C is reduced from i+1 to i, meaning fewer branches at t-1
        return C[0]
    else:
        for i in range(n, 0, -1): # Calculate value of option from backwards, last value is 1, which is value at time 0
            C_u = C[0:i] # Top i value
            C_d = C[1:i+1] # Bottom i value
            C = (P_u * C_u + P_d * C_d) / np.exp(r * dt) # nrow of C is reduced from i+1 to i, meaning fewer branches at t-1
            S_T = S * (u ** np.arange(i-1, -1, -1)) * (d ** np.arange(0, i)) # Calculate all possible stock prices at time t
            
            # Calculate call/ put payoff
            if isCall:
                S_T = np.maximum(S_T - K, np.zeros(i))
            else:
                S_T = np.maximum(K - S_T, np.zeros(i))
            
            # Compare values of early exercising and holding to maturity, choose the one with higher value
            C = np.maximum(C, S_T)
        return C[0]

In [8]:
Binomial_Pricer(100, 100, .05, 0, 1, .25, 1000)

12.333527192230251

<p>
    <font size=4>
        We can see the Binomial Tree model agrees with the Black-Scholes Model
<p/>

## Monte-Carlo Simulation
<p>
    <font size=4>
        The Monte Carlo Simulation is the third way of pricing derivatives. It can be used to price exotic options as well. The motivation of Monte Carlo simulation is to simulate different paths of stock price following Geometric Brownian Motion. Then, we plug those prices into the payoff function of the derivative. Finally, we calculate the average of discounted payoff to determine the current price of that dderivative. <br><br> 
        When we are simulating the stock prices, we can use the same set of Brownian motion {$B_t$} but flip the sign of it to generate a 'mirrored' set of prices. And we can use the 'mirrored' prices to get the 'averaged' payoff: $\frac{f(S)+f(S_{mirrored}  )}{2}$. This is called the 'Antithetic' method and it can decrease the variance of estimated derivative price. <br><br>
        The process of simulating stock prices from $t_i$ to $t_{i+1}$ is as follow:
<p/>
<p style='text-align: center;'>
    <font size=4>
        The stock price follows Geometric Brownian motion where <br>
        $dS_t=\mu S_tdt+\sigma S_tdB_t$ <br>
        and $S_{t_{i+1}}=S_{t_i}e^{(r-\frac{\sigma^2}{2})t+\sigma B_t}$ <br>
        where $t=\frac{T}{n}$, $T$ is the Year-to-maturity, $n$ is the simulating frequency per year (252 for daily) <br>
        $r$: risk-free rate <br>
        $\sigma$: annual volatility <br>
        $B_t$: Brownian Motion following $N(0,t)$
<p/>
<p>
    <font size=4>
        Note we cannot price American Option using Monte-Carlo because it is forward looking.
<p/>

In [23]:
def Monte_Carlo_Pricer(S, K, r, T, sig, n, seed=0, isCall=True):
    """
    This function finds the European option prices using Monte-Carlo Simulation
    input:
    S: Underlying asset spot price
    K: Strike price
    r: risk-free rate (if r = 4%, input 0.04)
    T: Trading day to maturity
    sig: standard deviation of stock return (if sig = 20%, input 0.2)
    n: number of path
    seed: random seed when generating the Brownian Motion
    isCall: True if call option, vice versa
    """
    np.random.seed(seed) # For getting same S_T for different trials
    Z_T = np.random.normal(0, 1, (n, T)) # Generate random normal variables with size 1000 x n
    Z_T_M = -Z_T # Reflect the Z_T for mirrored stock price

    dt = 1 / 252 # 252 trading days per year 
    t = np.tile(dt, (n, T)) # Create a 1000 x n matrix where all entries are dt

    W_t = Z_T * np.sqrt(dt) # Compute W_t for each price movement
    W_t_M = Z_T_M * np.sqrt(dt) # Compute W_t_M for each price movement
    
    index = (r - (sig**2)/2) * t + sig * W_t # Compute the index for each S(t_j+1) / S(t_j)
    Index = np.cumsum(index, axis=1) # Accumulate each index up to time t 
    index_M = (r - (sig**2)/2) * t + sig * W_t_M # Compute the index for each Mirrored S(t_j+1) / S(t_j)
    Index_M = np.cumsum(index_M, axis=1) # Accumulate each index_M up to time t 

    S_T = S * np.exp(Index) # Compute S_T
    S_T_M = S * np.exp(Index_M) # Compute Mirrored S_T
    
    last_S_T = S_T[:, -1] # Get the last value of S_T for each path
    last_S_T_M = S_T_M[:, -1] # Get the last value of S_T_M for each path
    if isCall:
        Payoff = np.maximum(last_S_T-K, 0)
        Payoff_M = np.maximum(last_S_T_M-K, 0)
    else:
        Payoff = np.maximum(K-last_S_T, 0)
        Payoff_M = np.maximum(K-last_S_T_M, 0)
    V = np.mean(np.exp(-r*T/252)*(Payoff+Payoff_M)/2)
    return V

In [22]:
Monte_Carlo_Pricer(100, 100, .05, 252, .25, 100000)

12.335719057287816

<p>
    <font size=4>
        Here we can see the Monte-Carlo agrees with Black-Scholes Model if number of path is sufficiently large.
<p/>